In [1]:
# Import module
import json
from web3 import Web3
from pathlib import Path
import streamlit as st
from openai import OpenAI
import requests
from PIL import Image
from io import BytesIO
import os

###############################################

json_headers = {
    "Content-Type": "application/json",
    "PINATA_API_KEY": os.getenv("PINATA_API_KEY"),
    "PINATA_SECRET_API_KEY": os.getenv("PINATA_SECRET_API_KEY"),
}

file_headers = {
    "PINATA_API_KEY": os.getenv("PINATA_API_KEY"),
    "PINATA_SECRET_API_KEY": os.getenv("PINATA_SECRET_API_KEY"),
}

def convert_data_to_json(content):
    data = {"pinataOptions": {"cidVersion": 1}, "pinataContent": content}
    return json.dumps(data)

def pin_file_to_ipfs(data):
    r = requests.post(
        "https://api.pinata.cloud/pinning/pinFileToIPFS",
        files={'file': data},
        headers=file_headers
    )
    print(r.json())
    ipfs_hash = r.json()["IpfsHash"]
    return ipfs_hash

def pin_json_to_ipfs(json):
    r = requests.post(
        "https://api.pinata.cloud/pinning/pinJSONToIPFS",
        data=json,
        headers=json_headers
    )
    print(r.json())
    ipfs_hash = r.json()["IpfsHash"]
    return ipfs_hash
##############################################

# # ############################################# use pinataS module in a different folder #####################
# import sys
# from importlib import import_module

# # Step 1: Add the folder to sys.path
# pinataS_path = './NFT_Minter_files'
# if pinataS_path not in sys.path:
#     sys.path.append(pinataS_path)

# # Step 2: Import the module using importlib
# pinata = 'pinataS'  # without the .py extension
# pinata_module = import_module(pinata)

# # Now, you can use the functions directly from the imported module
# pin_file_to_ipfs = pinata_module.pin_file_to_ipfs
# pin_json_to_ipfs = pinata_module.pin_json_to_ipfs
# convert_data_to_json = pinata_module.convert_data_to_json

# ##################################################################

# Define and connect a new Web3 provider
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

############## DALLE 3 ##########################

# Initialize OpenAI client with API key
client = OpenAI(api_key=st.secrets["OPENAI_API_KEY"])

generated_image_url = None  # Declare it globally to hold the generated image data

def generate_image(img_description):
    global generated_image_url
    try:
        img_response = client.images.generate(
            model="dall-e-3",
            prompt=img_description,
            size="1024x1024",
            quality="standard",
            n=1  # Always generate only one image
        )
        generated_image_url = img_response.data[0].url
        # st.write("Generated Image URL:", generated_image_url)  # For Debugging Purposes
    except Exception as e:
        st.error(f"Error generating image: {e}")

st.subheader("POWERED BY OPENAI & PINATA!!!!")
img_description = st.text_input("Enter a description for the image you want to generate:")

# Initialize session state
if 'image_content' not in st.session_state:
    st.session_state.image_content = None
if 'artwork_url' not in st.session_state:
    st.session_state.artwork_url = None

# Create a button to generate images
if st.button("Generate Image") and img_description:
    with st.spinner(text='Generating image...'):
        generate_image(img_description)
        
        # Fetch the image from the URL
        response = requests.get(generated_image_url)
        
        # Open the image
        image_content = Image.open(BytesIO(response.content))
        
        # Update session state with the new image content
        st.session_state.image_content = image_content

        # Save the generated image URL to session state
        st.session_state.artwork_url = generated_image_url
        
# Display the image
if st.session_state.image_content is not None:
    st.image(st.session_state.image_content, use_column_width=True)

################################################################################
# Load_Contract Function
################################################################################

@st.cache_resource
def load_contract():
    # Define the path to the ABI file
    abi_path = Path('./NFT_Minter_files/contracts/compiled/nft_registry_abi.json')
    
# Load the contract
contract = load_contract()

import json
from web3 import Web3
from pathlib import Path
import os
import streamlit as st

# Define and connect to a new Web3 provider
w3 = Web3(Web3.HTTPProvider(os.getenv("WEB3_PROVIDER_URI")))

def check_connection():
    if w3.is_connected():
        print("Connected to Ethereum network")
        return True
    else:
        print("Failed to connect to Ethereum network")
        return False

@st.cache_resource
def load_contract():
    print("Attempting to load the smart contract...")
    abi_path = Path('./NFT_Minter_files/contracts/compiled/nft_registry_abi.json')
    
    if not abi_path.exists():
        print("ABI file does not exist at:", abi_path)
        return None
    
    try:
        with abi_path.open() as f:
            contract_abi = json.load(f)
        print("ABI loaded successfully")

        contract_address = os.getenv("SMART_CONTRACT_ADDRESS")
        if not contract_address:
            print("SMART_CONTRACT_ADDRESS not set")
            return None
        print("Contract address:", contract_address)

        contract = w3.eth.contract(address=contract_address, abi=contract_abi)
        if contract:
            print("Contract instance created successfully")
            return contract
        else:
            print("Failed to create contract instance.")
            return None

    except FileNotFoundError:
        print("Contract ABI file not found.")
    except json.JSONDecodeError:
        print("Error decoding JSON.")
    except Exception as e:
        print("An unexpected error occurred:", str(e))
    return None


AttributeError: 'Web3' object has no attribute 'isConnected'

In [4]:
################ Looking for ABI

In [ ]:
# Import module
import json
from web3 import Web3
from pathlib import Path
import streamlit as st
from openai import OpenAI
import requests
from PIL import Image
from io import BytesIO
import os

################################################################################
# Load_Contract Function
################################################################################

@st.cache_resource

def load_contract():
    # Define the path to the ABI file
    abi_path = Path('./NFT_Minter_files/contracts/compiled/nft_registry_abi.json')
    if not abi_path.exists():
        raise FileNotFoundError(f"The ABI file was not found at {abi_path}")

    # Load the ABI
    with open(abi_path) as file:
        abi = json.load(file)

    # Ensure the WEB3_PROVIDER_URI is correctly set
    web3_provider_uri = os.getenv("WEB3_PROVIDER_URI")
    if web3_provider_uri is None:
        raise EnvironmentError("WEB3_PROVIDER_URI environment variable not set")

    # Connect to Web3
    w3 = Web3(Web3.HTTPProvider(web3_provider_uri))
    if not w3.isConnected():
        raise ConnectionError("Failed to connect to the Ethereum client")

    # Load the contract
    contract_address = os.getenv("CONTRACT_ADDRESS")  # Make sure this is set in your env
    if contract_address is None:
        raise EnvironmentError("CONTRACT_ADDRESS environment variable not set")

    contract = w3.eth.contract(address=contract_address, abi=abi)
    return contract

# Load the contract
contract = load_contract()



In [6]:
import os
print("Current working directory:", os.getcwd())

Current working directory: /home/jiaoyinyang/Documents/UMN_FinTech/UofM-VIRT-FIN-PT-11-2023-U-LOLC-main/01-Sessions/22-dApps/2/Activities/06-Evr_Art_Registry_with_IPFS/Solved/ArtRegistry/NFT_APP/pages
